This notebook is to gather PPP data (2021 loans through March 31), join with BLS QCEW data, and calculate PPP penetration for display on ArcGIS as requested by the SBA

Initial settings; Load PPP data

In [1]:
import loans_common2021 as co

import pandas as pd
import zipfile
from plotnine import *    # python lib to use ggplot
from io import BytesIO
from urllib.request import urlopen

pd.set_option('max_columns', 100)

naics = 'NAICS2'
excsole = True

fpath = '/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/'\
                                                    'FOIA 2021-04-01/'
loans = co.ReadPPPdata(naics, fpath)

/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_9.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_8.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_150k_plus.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_3.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_2.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_1.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_5.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_4.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_6.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k

### Match draws going to the same business

In [52]:
import loans_common2021 as co
import datetime
print(datetime.datetime.now().time())
tmp = co.MatchLoanDraws(loans)   # matches loans using Probabilistic record linkage with Business name and address

18:05:04.900829
ProcessingMethod ['PPP' 'PPS']


ValueError: index of DataFrame is not unique

In [28]:
loansmatch.to_csv('/Users/aligo/Downloads/FEMA recovery data/PPPloansNE2021-MatchedDraws.csv')

match county information wherever possible

In [80]:

loans21 = co.FilterLoans(loans)
[loans_y, countyfips] = co.MatchCounties(loans21, excsole, naics)

/Users/aligo/git-repos/FEMA/loans_common2021.py:109: FutureWarning: The default value of regex will change from True to False in a future version.


unique cities in loan dataset: 2293
Unique matches: 1383
Unique non-matches: 910
loans in loan dataset: 156906
loans with matches: 136978
loans non-matches: 19928
zip-county pairs in New Englad: 2496
zip-county pairs in New England, deduplicated: 2206
loans in loan dataset: 156906
1st draw loans:  41480
2nd draw loans:  115426
After ZIP match: loans with matches:  156903
After ZIP match: loans non-matches:  3
MANUALLY FIX THE COUNTIES IN EXCEL FILE /Users/aligo/Downloads/FEMA recovery data/ppp addresses MISSnosole2021.xlsx
 AND CHANGE ITS NAME FROM MISS TO MANUAL


### EXECUTE THIS COMMAND ONLY AFTER CHECKING THE EXCEL FILE OF UNMATCHED LOANS in Downloads/

In [ ]:
loansum = co.AddManualCounties(loans_y, excsole, naics)

In [ ]:

# TOTAL NUMBER OF BUSINESSES - from US Census CBP
url = urlopen("https://www2.census.gov/programs-surveys/cbp/datasets/2018/cbp18co.zip")
#Download Zipfile and create pandas DataFrame
zipfile = zipfile.ZipFile(BytesIO(url.read()))
cbp = pd.read_csv(zipfile.open('cbp18co.txt'), na_values='N')

cbp['FIPST'] = cbp['fipstate'].astype(str).str.pad(2,fillchar='0')
cbp['area_fips'] = cbp['FIPST'] + cbp['fipscty'].astype(str).str.pad(3,fillchar='0')
cnt = cbp[cbp['FIPST'].isin(co.NEstfips)    # New England counties
          & ~cbp['fipscty'].eq(999)      # exclude fipscty = 999 that are "statewide" totals
          & cbp['naics'].str.contains('^[0-9][0-9]----',regex=True)]    # 2-digit NAICS codes
cnt = cnt.assign( industry_code = cnt['naics'].str.slice(start=0,stop=2) )    # 2-digit NAICS codes, cleaned
cnt['NEstabs'] = cnt[['n<5','n5_9','n10_19','n20_49','n50_99','n100_249','n250_499']
                     ].sum(axis=1, skipna=True)         # num Establishments with < 500 employees
cnt = cnt[['area_fips','industry_code','NEstabs']].set_index('area_fips')
# Add state and county name
cnt = cnt.join(countyfips.set_index('COUNTY'))

# Join Total Num businesses + Num of loans
cnt = cnt.reset_index()
cnt.columns = ['COUNTYfips','NAICS2','NEstabs','State','COUNTYName']
# adjusts 2-digit NAICS that are joint, e.g. NAICS 31-33 Manufacturing
cnt = co.OverrideNAICS2(cnt)

pen = co.CalcPenetration(loansum, cnt, naics)

# total per county 
dfc = pen.groupby(['COUNTYName']).agg('sum').reset_index()
dfc['penetration'] = dfc['NLoans'] / dfc['NEstabs']
ggplot(dfc, aes(x='reorder(COUNTYName,penetration)', y='penetration')
          ) + geom_bar(stat="identity"
          ) + xlab('County'
          ) + ylab('PPP Loan Penetration'
          ) + ggtitle('New England PPP Penetrations per County'
          ) + theme(axis_text_y = element_text(size=6)
          ) + coord_flip()

# total per NAICS
dfp = pen.groupby(['NAICS2','NAICSdescr']).agg('sum').reset_index()
dfp['penetration'] = dfp['NLoans'] / dfp['NEstabs']

ggplot(dfp, aes(x='reorder(NAICSdescr,penetration)', y='penetration')
          ) + geom_bar(stat="identity"
          ) + xlab('NAICS 2 Digit Sector'
          ) + ylab('PPP Loan Penetration'
          ) + ggtitle('New England PPP Penetrations per NAICS Sector'
#          ) + scale_y_continuous(trans = 'log2'
          ) + coord_flip()

# Histogram of County-NAICS penetrations
ggplot(pen[pen.penetration.le(1)], aes(x='penetration')
       ) + geom_histogram(binwidth=.05
       ) + xlab('PPP Loan Penetration'
       ) + ylab('Number of Counties-Sectors'
       ) + ggtitle('Distribution of PPP Penetration <= 1 in New England')

# outlier penetrations
ggplot(pen, aes(x='State', y='penetration')
       ) + geom_boxplot(
       ) + xlab('State'
       ) + ylab('Penetration'
       ) + ggtitle('Outliers of PPP Penetration in New England')





